In [35]:
import json
import requests
import pandas as pd
from pathlib import Path
import hvplot.pandas
import panel as pn
import holoviews as hv
import numpy as np
from pathlib import Path
from sklearn import cluster, covariance, manifold
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

import warnings
warnings.filterwarnings('ignore')


In [36]:
events=pd.read_csv(
    Path('Institutional_Adoption_Events_negative_in_2021.csv'),
    index_col='DATE',
    parse_dates=True,
    infer_datetime_format=True
)
events=events.fillna(0)
events

,NEWS,CATEGORY,Institutional Adopt
DATE,,,
2021-01-11,UK FCA Warns: Prepare to Lose All Your Money,Government,-1
2021-01-25,UK FCA Warns: Bitcoin's Days Are Numbered,Government,-1
2021-04-22,Biden Tax Hike Proposal,Government,-1
2021-04-23,Biden Capital Gains Tax Rumors,Government,-1
2021-05-12,Musk Tweet: Concerns About BTC Energy Use,Musk Tweet,-1
2021-05-16,Musk Tweet: Tesla May Sell Its Crypto,Musk Tweet,-1
2021-05-18,China Bans Institutions From Crypto Business,China,-1
2021-05-20,US Treasury Calls For Stricter Crypto Compliance,Government,-1
2021-05-21,China Crackdown on Mining & Trading,China,-1


In [37]:
events=events.drop(['NEWS', 'CATEGORY'], axis=1)
events.head()

,Institutional Adopt
DATE,
2021-01-11,-1
2021-01-25,-1
2021-04-22,-1
2021-04-23,-1
2021-05-12,-1


In [38]:
prices=pd.read_csv(
    Path('crypto_market_caps.csv'),
    index_col='date',
    parse_dates=True,
    infer_datetime_format=True

)
prices.head()

,spell-token,illuvium,polyswarm,hackenai,mercurial,bitcoin,ethereum,solana,polkadot,cardano,...,uniswap,antimatter,1inch,aave,maker,lido-dao,serum,audius,arweave,bonfida
date,,,,,,,,,,,,,,,,,,,,,
2021-05-29,0.000000,-0.087474,-0.084921,-0.178955,-0.069173,-0.075526,-0.116392,-0.141745,-0.110146,-0.081747,...,-0.086776,-0.107659,-0.161449,-0.144139,-0.127478,-0.204751,-0.148758,-0.141004,-0.074100,-0.154125
2021-05-30,0.231838,-0.029180,-0.077416,-0.050995,-0.020782,-0.021363,-0.048031,-0.045486,-0.063362,-0.056490,...,-0.101236,-0.070963,-0.058184,-0.062235,-0.049803,-0.049462,-0.048317,-0.080900,-0.092101,-0.066768
2021-05-31,0.238859,0.067797,0.092618,-0.037861,0.022546,0.022048,0.035723,0.030508,0.015360,0.099605,...,0.095878,0.008402,0.111175,0.081288,0.070708,0.119675,0.043991,0.033885,0.081935,0.058975
2021-06-01,-0.059507,0.106957,0.026690,-0.004131,-0.004840,0.045575,0.130062,0.145793,0.143006,0.109167,...,0.102625,0.202528,0.106781,0.178813,0.082806,0.195291,0.104627,0.063740,0.081550,0.123031
2021-06-02,-0.205798,-0.007821,-0.032677,0.333030,-0.024114,-0.014893,-0.025359,-0.050999,-0.015254,0.000810,...,-0.030430,-0.194083,-0.023898,-0.023796,0.002862,-0.057441,-0.031295,0.050371,0.002730,-0.042151


In [39]:
combined=pd.merge(prices,events,left_index=True, right_index=True)
combined.head()

,spell-token,illuvium,polyswarm,hackenai,mercurial,bitcoin,ethereum,solana,polkadot,cardano,...,antimatter,1inch,aave,maker,lido-dao,serum,audius,arweave,bonfida,Institutional Adopt
2021-06-03,0.233313,0.048309,-0.086979,0.064020,0.009435,0.022948,0.028792,0.100485,0.140927,0.009557,...,0.055345,0.120798,0.032459,-0.004457,0.073983,0.065638,0.220224,0.100321,0.071874,-1
2021-06-21,-0.040450,0.024112,-0.068335,-0.034691,-0.020246,0.005598,0.031893,-0.001046,0.017024,0.030325,...,-0.065572,0.141981,0.071804,0.051422,0.001251,0.009227,-0.013097,0.011193,-0.003792,-1
2021-06-28,0.094486,0.393203,-0.006242,-0.006441,0.002675,0.081679,0.092218,0.086898,0.046501,0.078923,...,0.017638,0.076903,0.095209,0.066262,0.112646,0.070818,0.055992,0.141560,0.085336,-1
2021-07-08,0.385629,-0.016118,-0.042761,0.026644,0.005079,-0.006494,-0.002147,0.069729,0.054552,-0.008347,...,-0.051359,-0.025954,-0.010753,0.035841,0.071633,-0.040167,0.081559,0.076101,0.049498,-1
2021-09-27,0.031190,-0.000622,0.065373,0.037680,-0.021126,0.021109,0.036512,-0.005338,-0.043229,-0.024846,...,0.463079,0.071783,0.059910,0.046358,0.068655,0.004005,-0.012441,0.000000,0.002621,-1


In [40]:
combined = combined.dropna(axis=0, how='all')
combined = combined.dropna(axis=1)
combined

,spell-token,illuvium,polyswarm,hackenai,mercurial,bitcoin,ethereum,solana,polkadot,cardano,...,antimatter,1inch,aave,maker,lido-dao,serum,audius,arweave,bonfida,Institutional Adopt
2021-06-03,0.233313,0.048309,-0.086979,0.064020,0.009435,0.022948,0.028792,0.100485,0.140927,0.009557,...,0.055345,0.120798,0.032459,-0.004457,0.073983,0.065638,0.220224,0.100321,0.071874,-1
2021-06-21,-0.040450,0.024112,-0.068335,-0.034691,-0.020246,0.005598,0.031893,-0.001046,0.017024,0.030325,...,-0.065572,0.141981,0.071804,0.051422,0.001251,0.009227,-0.013097,0.011193,-0.003792,-1
2021-06-28,0.094486,0.393203,-0.006242,-0.006441,0.002675,0.081679,0.092218,0.086898,0.046501,0.078923,...,0.017638,0.076903,0.095209,0.066262,0.112646,0.070818,0.055992,0.141560,0.085336,-1
2021-07-08,0.385629,-0.016118,-0.042761,0.026644,0.005079,-0.006494,-0.002147,0.069729,0.054552,-0.008347,...,-0.051359,-0.025954,-0.010753,0.035841,0.071633,-0.040167,0.081559,0.076101,0.049498,-1
2021-09-27,0.031190,-0.000622,0.065373,0.037680,-0.021126,0.021109,0.036512,-0.005338,-0.043229,-0.024846,...,0.463079,0.071783,0.059910,0.046358,0.068655,0.004005,-0.012441,0.000000,0.002621,-1
2021-04-22,0.000000,-0.063309,-0.044171,0.427815,0.000000,-0.040228,0.017965,0.023371,-0.039815,-0.034160,...,-0.055628,0.012961,-0.014037,0.161273,-0.047253,0.007571,-0.070632,-0.023724,0.022215,-1
2021-04-23,0.000000,0.014913,-0.152139,-0.161849,0.000000,-0.041004,0.023391,0.193001,-0.024329,-0.050448,...,-0.075863,-0.077949,0.038496,0.023060,-0.032614,0.103431,-0.018742,-0.038372,0.129322,-1
2021-05-12,0.000000,0.019192,0.050507,-0.073835,0.000000,0.018436,0.044118,0.041909,0.050372,0.064121,...,-0.045710,0.045803,0.051394,0.074398,0.003301,0.050127,0.021041,0.207929,0.052756,-1
2021-05-16,0.000000,-0.096664,-0.051121,-0.040620,0.000000,-0.064790,-0.101345,0.009458,-0.081225,0.082550,...,-0.117898,-0.081861,-0.133378,-0.080291,-0.091162,-0.100104,-0.102155,-0.095139,-0.039170,-1
2021-05-18,0.000000,-0.004831,-0.032908,0.126400,0.000000,-0.060157,-0.079908,0.007466,-0.077167,-0.105774,...,-0.047775,-0.059087,0.127019,-0.053584,-0.786075,-0.044020,0.134674,-0.058431,-0.032635,-1


In [41]:
combined_new=combined.drop(['Institutional Adopt'], axis=1)
combined_new

,spell-token,illuvium,polyswarm,hackenai,mercurial,bitcoin,ethereum,solana,polkadot,cardano,...,uniswap,antimatter,1inch,aave,maker,lido-dao,serum,audius,arweave,bonfida
2021-06-03,0.233313,0.048309,-0.086979,0.064020,0.009435,0.022948,0.028792,0.100485,0.140927,0.009557,...,0.009598,0.055345,0.120798,0.032459,-0.004457,0.073983,0.065638,0.220224,0.100321,0.071874
2021-06-21,-0.040450,0.024112,-0.068335,-0.034691,-0.020246,0.005598,0.031893,-0.001046,0.017024,0.030325,...,0.047777,-0.065572,0.141981,0.071804,0.051422,0.001251,0.009227,-0.013097,0.011193,-0.003792
2021-06-28,0.094486,0.393203,-0.006242,-0.006441,0.002675,0.081679,0.092218,0.086898,0.046501,0.078923,...,0.068190,0.017638,0.076903,0.095209,0.066262,0.112646,0.070818,0.055992,0.141560,0.085336
2021-07-08,0.385629,-0.016118,-0.042761,0.026644,0.005079,-0.006494,-0.002147,0.069729,0.054552,-0.008347,...,0.003621,-0.051359,-0.025954,-0.010753,0.035841,0.071633,-0.040167,0.081559,0.076101,0.049498
2021-09-27,0.031190,-0.000622,0.065373,0.037680,-0.021126,0.021109,0.036512,-0.005338,-0.043229,-0.024846,...,0.212313,0.463079,0.071783,0.059910,0.046358,0.068655,0.004005,-0.012441,0.000000,0.002621
2021-04-22,0.000000,-0.063309,-0.044171,0.427815,0.000000,-0.040228,0.017965,0.023371,-0.039815,-0.034160,...,0.050706,-0.055628,0.012961,-0.014037,0.161273,-0.047253,0.007571,-0.070632,-0.023724,0.022215
2021-04-23,0.000000,0.014913,-0.152139,-0.161849,0.000000,-0.041004,0.023391,0.193001,-0.024329,-0.050448,...,0.027996,-0.075863,-0.077949,0.038496,0.023060,-0.032614,0.103431,-0.018742,-0.038372,0.129322
2021-05-12,0.000000,0.019192,0.050507,-0.073835,0.000000,0.018436,0.044118,0.041909,0.050372,0.064121,...,0.049986,-0.045710,0.045803,0.051394,0.074398,0.003301,0.050127,0.021041,0.207929,0.052756
2021-05-16,0.000000,-0.096664,-0.051121,-0.040620,0.000000,-0.064790,-0.101345,0.009458,-0.081225,0.082550,...,-0.087573,-0.117898,-0.081861,-0.133378,-0.080291,-0.091162,-0.100104,-0.102155,-0.095139,-0.039170
2021-05-18,0.000000,-0.004831,-0.032908,0.126400,0.000000,-0.060157,-0.079908,0.007466,-0.077167,-0.105774,...,-0.075334,-0.047775,-0.059087,0.127019,-0.053584,-0.786075,-0.044020,0.134674,-0.058431,-0.032635


In [42]:
variation = combined_new
variation.shape

(14, 52)

In [43]:
column_names= list(combined_new)

In [44]:
# calculate mean and variance for each token
annual_returns = []
variance = []
for x in column_names:
    annual_returns.append(combined_new[x].mean()*252)
    variance.append(combined_new[x].std()*np.sqrt(252))
    
    

In [45]:
stats=pd.DataFrame([annual_returns,variance])

In [46]:
stats_df=stats.T

In [47]:
stats_df.columns=('annual_return','variance')

In [48]:
stats_df.index=column_names

In [49]:
stats_df.head()

,annual_return,variance
spell-token,12.675029,1.864122
illuvium,3.189154,2.080286
polyswarm,-10.577563,1.456084
hackenai,5.199850,2.403955
mercurial,42.133995,10.042468


In [50]:
# Create a list to store inertia values and the values of k
inertia = []
k = list(range(1, 11))

In [51]:
# Create a for-loop where each value of k is evaluated using the K-means algorithm
# Fit the model using the service_ratings DataFrame
# Append the value of the computed inertia from the `inertia_` attribute of the KMeans model instance
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(stats_df)
    inertia.append(k_model.inertia_)

In [52]:
# Define a DataFrame to hold the values for k and the corresponding inertia
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

# Review the DataFrame
df_elbow.head()

,k,inertia
0,1,3819.100285
1,2,1832.951898
2,3,688.059518
3,4,338.314023
4,5,210.211339


In [53]:
# Plot the DataFrame
df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)

:Curve   [k]   (inertia)

In [54]:
# Create and initialize the K-means model instance for 4 clusters
model = KMeans(n_clusters=4, random_state=1)

In [55]:
# Fit the data to the instance of the model
model.fit(stats_df)

# Make predictions about the data clusters using the trained model
crypto_clusters = model.predict(stats_df)
crypto_clusters

array([3, 0, 2, 0, 1, 2, 0, 3, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 0,
       0, 3, 0, 2, 2, 2, 0, 0, 2, 0, 0, 2, 2, 2, 0, 2, 2, 0, 0, 2, 3, 2,
       0, 3, 0, 0, 2, 0, 3, 3], dtype=int32)

In [56]:
len(crypto_clusters)

52

In [57]:
crypto_clusters_df = pd.DataFrame(crypto_clusters)
crypto_clusters_df.head()

,0
0,3
1,0
2,2
3,0
4,1


In [58]:
stats_df['crypto_cluster']=crypto_clusters
stats_df['names']=column_names
stats_df


,annual_return,variance,crypto_cluster,names
spell-token,12.675029,1.864122,3,spell-token
illuvium,3.189154,2.080286,0,illuvium
polyswarm,-10.577563,1.456084,2,polyswarm
hackenai,5.199850,2.403955,0,hackenai
mercurial,42.133995,10.042468,1,mercurial
bitcoin,-2.577197,0.926795,2,bitcoin
ethereum,0.810657,1.487722,0,ethereum
solana,8.214550,2.094857,3,solana
polkadot,-5.251681,1.847720,2,polkadot
cardano,-1.754249,1.542801,2,cardano


In [59]:
# Plot the crypto clusters Data Frame
stats_df.hvplot.scatter(x="annual_return", y="variance", by="crypto_cluster",hover=True, hover_cols=['index'])
#stats_df.hvplot.labels(x='annual_return', y='variance',by='crypto_cluster', text='names',text_baseline='top', hover=True)

:NdOverlay   [crypto_cluster]
   :Scatter   [annual_return]   (variance,index)

In [60]:
correlations_df=combined_new.corr()
btc_correlations=correlations_df.loc['bitcoin']

In [61]:
btc_correlations=btc_correlations.sort_values()

btc_correlations.hvplot.scatter()

:Scatter   [index]   (bitcoin)

In [62]:
plt.figure(figsize = (16,10))
ax = sns.heatmap(correlations_df, annot=True, linewidths=.5)


In [63]:
correlations_df

,spell-token,illuvium,polyswarm,hackenai,mercurial,bitcoin,ethereum,solana,polkadot,cardano,...,uniswap,antimatter,1inch,aave,maker,lido-dao,serum,audius,arweave,bonfida
spell-token,1.000000,0.161467,-0.021516,0.054725,-0.117178,0.204216,0.085180,0.176055,0.399292,0.063145,...,-0.069673,0.073808,0.060773,-0.052195,0.049715,0.091107,0.050568,0.344257,0.176754,-0.018637
illuvium,0.161467,1.000000,0.301272,0.100441,0.143418,0.794536,0.678691,0.571080,0.628053,0.557639,...,0.256877,0.349267,0.520973,0.552562,0.508392,0.283037,0.624777,0.556481,0.558663,0.361429
polyswarm,-0.021516,0.301272,1.000000,0.309880,-0.024285,0.512493,0.641141,0.271089,0.385680,0.472375,...,0.829120,0.526000,0.751709,0.683491,0.527730,0.509703,0.534952,0.298418,0.400957,0.161486
hackenai,0.054725,0.100441,0.309880,1.000000,0.155665,0.221413,0.349711,0.309308,0.348867,0.215991,...,0.231922,0.294048,0.386976,0.337214,0.631522,0.059404,0.366479,0.384258,0.181810,0.258088
mercurial,-0.117178,0.143418,-0.024285,0.155665,1.000000,0.486436,0.268938,0.425309,0.278757,0.518105,...,0.030929,0.180633,0.110918,0.107049,0.217364,0.208928,0.328088,0.458597,0.122750,0.927336
bitcoin,0.204216,0.794536,0.512493,0.221413,0.486436,1.000000,0.858930,0.692485,0.816385,0.811338,...,0.463808,0.592630,0.740677,0.608701,0.694562,0.541674,0.789008,0.718057,0.617976,0.694165
ethereum,0.085180,0.678691,0.641141,0.349711,0.268938,0.858930,1.000000,0.782720,0.819913,0.707519,...,0.744889,0.563762,0.900168,0.835921,0.871899,0.712771,0.937354,0.662085,0.705532,0.568318
solana,0.176055,0.571080,0.271089,0.309308,0.425309,0.692485,0.782720,1.000000,0.861076,0.729289,...,0.370305,0.414895,0.604761,0.642831,0.716206,0.397064,0.921824,0.811735,0.558651,0.721357
polkadot,0.399292,0.628053,0.385680,0.348867,0.278757,0.816385,0.819913,0.861076,1.000000,0.767363,...,0.368595,0.462141,0.758619,0.636358,0.743927,0.439363,0.868091,0.870257,0.701077,0.575903
cardano,0.063145,0.557639,0.472375,0.215991,0.518105,0.811338,0.707519,0.729289,0.767363,1.000000,...,0.367251,0.353252,0.648149,0.426445,0.623462,0.499897,0.738482,0.596058,0.473146,0.714059


In [64]:
# Plot the DataFrame
# Visualization
plt.figure(1, facecolor='w', figsize=(12, 9))
plt.clf()
ax = plt.axes([0., 0., 1., 1.])
plt.axis('off')


#Display a graph of the partial correlations
partial_correlations = edge_model.precision_.copy()
d = 1 / np.sqrt(np.diag(partial_correlations))
partial_correlations *= d
partial_correlations *= d[:, np.newaxis]
non_zero = (np.abs(np.triu(partial_correlations, k=1)) > 0.02)


# Plot the nodes using the coordinates of our embedding
plt.scatter(embedding[0], embedding[1], s=100 * d ** 2, c=labels,
            cmap=plt.cm.nipy_spectral)

# Plot the edges
start_idx, end_idx = np.where(non_zero)

# a sequence of (*line0*, *line1*, *line2*), where::
#            linen = (x0, y0), (x1, y1), ... (xm, ym)
segments = [[embedding[:, start], embedding[:, stop]]
            for start, stop in zip(start_idx, end_idx)]
values = np.abs(partial_correlations[non_zero])
lc = LineCollection(segments,
                    zorder=0, cmap=plt.cm.hot_r,
                    norm=plt.Normalize(0, .7 * values.max()))
lc.set_array(values)
lc.set_linewidths(15 * values)
ax.add_collection(lc)


# Add a label to each node. The challenge here is that we want to
# position the labels to avoid overlap with other labels
for index, (name, label, (x, y)) in enumerate(
        zip(names, labels, embedding.T)):
    dx = x - embedding[0]
    dx[index] = 1
    dy = y - embedding[1]
    dy[index] = 1
    this_dx = dx[np.argmin(np.abs(dy))]
    this_dy = dy[np.argmin(np.abs(dx))]
    if this_dx > 0:
        horizontalalignment = 'left'
        x = x + .002
    else:
        horizontalalignment = 'right'
        x = x - .002
    if this_dy > 0:
        verticalalignment = 'bottom'
        y = y + .002
    else:
        verticalalignment = 'top'
        y = y - .002
    plt.text(x, y, name, size=10,
             horizontalalignment=horizontalalignment,
             verticalalignment=verticalalignment,
             bbox=dict(facecolor='w',
                       edgecolor=plt.cm.nipy_spectral(label / float(n_labels)),
                       alpha=.6))
plt.xlim(embedding[0].min() - .15 * embedding[0].ptp(),
         embedding[0].max() + .10 * embedding[0].ptp(),)
plt.ylim(embedding[1].min() - .03 * embedding[1].ptp(),
         embedding[1].max() + .03 * embedding[1].ptp())
plt.show()

NameError: name 'edge_model' is not defined